In [1]:
!pip install openpyxl


In [ ]:
import pandas as pd
import boto3
import time
import re
from botocore.exceptions import ClientError
from tqdm import tqdm

print('Running boto3 version:', boto3.__version__)

# AWS Bedrock setup
region = 'us-west-2'
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)
model_id = "meta.llama3-70b-instruct-v1:0"

# Load Excel Data
df = pd.read_excel("full.xlsx")

def invoke_bedrock_model(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            response = client.converse(
                modelId=id,
                messages=[{"role": "user", "content": [{"text": prompt}]}],
                inferenceConfig={
                    "temperature": temperature,
                    "maxTokens": max_tokens,
                    "topP": top_p
                }
            )
            result = response['output']['message']['content'][0]['text']
            return result
        except ClientError as e:
            if e.response['Error']['Code'] == 'ThrottlingException':
                wait_time = 2 ** retries
                time.sleep(wait_time)
                retries += 1
            else:
                print(f"ClientError: {e}")
                break
        except Exception as e:
            print(f"Unexpected error: {e}")
            break
    return None

prompt_template = """
Extract relevant attributes from the provided product description, considering synonyms, related terms, and contextual meaning. Assign a binary value (1 if the attribute is explicitly mentioned or strongly implied, 0 if absent) for each attribute in the exact order listed below.
Product Description:
{product_description}
Attributes to Extract:
dark_pigmentation
acne
eye_contour
homogeneity
lack_firmness
lack_radiance
pores
fine_lines
wrinkles_fine-lines
eye-wrinkles
undereye-bags
generic
Age-18-34
Age-35-54
Age-55-99
dry
normal
oily
combination
sensitivity-high
sensitivity-low
no_sensitivity
male
female
cleanse
prepare
treat
targeted
care
moisturize
protect
day
night
Extraction Guidelines:
Contextual & Synonym-Based Mapping:
Dark Pigmentation → Hyperpigmentation, dark spots, discoloration, uneven tone, etc.
Acne → Pimples, blemishes, breakouts, clogged pores, etc.
Eye Contour → Dark circles, puffiness, under-eye concerns, tired eyes, etc.
Fine Lines & Wrinkles → Crow’s feet, aging lines, visible aging, etc.
Undereye Bags → Puffiness, tired eyes, sagging skin under eyes, etc.
Ingredient-Based Recognition:
Niacinamide, Vitamin C, Licorice Extract → Brightening, dark spots, pigmentation, etc.
Salicylic Acid, Benzoyl Peroxide (BPO), Tea Tree Oil → Acne treatment, blemishes, excess oil, etc.
Hyaluronic Acid, Ceramides, Peptides → Moisturization, hydration, skin barrier repair, etc.
Retinol, Peptides, Collagen, CoQ10 → Anti-aging, wrinkle treatment, firming, fine line reduction, etc.
Caffeine, Green Tea Extract → Reduces puffiness, undereye bags, antioxidants, etc.
Vitamin A, B, C, E → Brightening, radiance, anti-aging, pigmentation improvement, etc.
SPF, Zinc Oxide, Titanium Dioxide → UV protection, sun defense, sunscreen, etc.
Age Group Classification:
18-34 → Acne prevention, hydration, early signs of aging, etc.
35-54 → Wrinkle reduction, collagen boosting, firming, etc.
55-99 → Mature skin, deep wrinkles, loss of elasticity, hormonal skin changes, etc.
Skin Type Classification:
Dry → Hydration, nourishment, intense moisture, etc.
Oily → Mattifying, oil control, absorbs excess oil, etc.
Combination → Balances oily and dry areas, etc.
Sensitivity Classification:
High Sensitivity → Soothing, calming, fragrance-free, anti-inflammatory, etc.
Low Sensitivity → Mild formulations, balanced ingredients, etc.
No Sensitivity → Strong active ingredients, retinol, exfoliants, etc.
Function-Based Identification:
Cleanse → Cleansing, purifying, face wash, etc.
Prepare → Prepping skin, toner, exfoliation, etc.
Treat → Serums, acne treatments, anti-aging solutions, etc.
Targeted → Specific concerns like dark spots, wrinkles, acne, etc.
Care → General skincare maintenance, nourishment, etc.
Moisturize → Hydration, nourishment, skin barrier repair, etc.
Protect → SPF, UV protection, environmental defense, etc.
Day/Night Usage → Morning routine (day), overnight repair (night), etc.
Output Format:
Return only a comma-separated list of 1s and 0s corresponding to the attributes in the specified order. Do not provide explanations.
"""
attribute_names = [
    "dark_pigmentation", "acne", "eye_contour", "homogeneity", "lack_firmness",
    "lack_radiance", "pores", "fine_lines", "wrinkles_fine-lines", "eye-wrinkles",
    "undereye-bags", "generic", "18-34", "35-54", "55-99", "dry", "normal", "oily",
    "combination", "sensitivity-high", "sensitivity-low", "no_sensitivity", "male",
    "female", "cleanse", "prepare", "treat", "targeted", "care", "moisturize", "protect",
    "day", "night"
]

results = []
total_rows = len(df)

with tqdm(total=total_rows, desc="Total Progress", position=0) as total_bar:
    for index, row in enumerate(df.itertuples(index=False), start=1):
        tqdm.write(f"Processing Row: {index}/{total_rows}", end="\r")  # Dynamically update row number
        product_description = row.text_raw
        prompt = prompt_template.format(product_description=product_description)
        output = invoke_bedrock_model(bedrock, model_id, prompt)
        
        if output:
            output_values = re.findall(r'\b[01]\b', output)
            result = {"text_raw": product_description}
            output_values += ['0'] * (len(attribute_names) - len(output_values))
            for i, attr in enumerate(attribute_names):
                result[attr] = int(output_values[i])
            results.append(result)
        else:
            print(f"Error processing row {index}: No output returned.")
        
        total_bar.update(1)

results_df = pd.DataFrame(results)
output_path = 'fullmeta.xlsx'
results_df.to_excel(output_path, index=False)
print(f"\nResults saved to {output_path}")

Running boto3 version: 1.36.3


Total Progress:   0%|          | 0/6240 [00:00<?, ?it/s]

Total Progress:   0%|          | 1/6240 [00:01<3:04:18,  1.77s/it]

Total Progress:   0%|          | 2/6240 [00:03<3:24:01,  1.96s/it]

Total Progress:   0%|          | 3/6240 [00:05<3:00:21,  1.73s/it]

...

Total Progress:  22%|██▏       | 1352/6240 [1:12:31<6:35:27,  4.85s/it]

Total Progress:  22%|██▏       | 1353/6240 [1:12:33<5:34:25,  4.11s/it]

Total Progress:  22%|██▏       | 1354/6240 [1:12:35<4:41:47,  3.46s/it]

Total Progress:  22%|██▏       | 1355/6240 [1:12:37<4:03:32,  2.99s/it]

Total Progress:  22%|██▏       | 1356/6240 [1:12:39<3:35:44,  2.65s/it]